In [2]:
import numpy as np
from collections import deque
from numpy.lib.stride_tricks import as_strided

In [25]:
with open("../data/14.txt") as f:
    data = [line.rstrip() for line in f.readlines()]

if len(data)<50:
    nc=11
    nr=7
else:
    nc=101
    nr=103



In [32]:
robots = []
for line in data:
    p,v = line.split()
    p = np.array([int(pp) for pp in p.replace("p=","").split(",")][::-1])
    v = np.array([int(vv) for vv in v.replace("v=","").split(",")][::-1])
    robots.append((p,v))
robots = deque(robots)

In [33]:
grid = np.zeros((nr,nc))

# initialize robot positions
for robot in robots:
    pos,v = robot
    grid[*pos] += 1

num_robots = len(robots)
num_robots

500

In [34]:
np.sqrt(500)

np.float64(22.360679774997898)

In [35]:
def detect_trees(grid, sub_shape=(8,8), threshold = 20):
    view_shape = tuple(np.subtract(grid.shape, sub_shape) + 1) + sub_shape
    arr_view = as_strided(grid, view_shape, grid.strides * 2)
    arr_view = arr_view.reshape((-1,) + sub_shape)

    poss_windows = 0
    for w in arr_view:
        if w.sum()>=threshold:
            poss_windows +=1
    return poss_windows

def output_image(grid,fn,nr=nr,nc=nc):
    lines = ""
    for r in range(nr):
        for c in range(nc):
            lines += "X" if grid[r,c]>0 else " "
        lines += "\n"
    with open(fn,"w") as f:
        f.writelines(lines)


In [36]:
def detect_center_mass(grid):
    return grid[21:69,20:-20].sum()>250

In [ ]:
num_steps = 100
for step in range(num_steps):
    c = 0
    while c < num_robots:
        robot = robots.popleft()
        pos,v = robot
        new_pos = pos + v
        new_pos = np.mod(new_pos,(nr,nc))
        grid[*pos] -= 1 
        grid[*new_pos] += 1
        robot = (new_pos,v)
        robots.append(robot)
        c +=1
    
    if detect_center_mass(grid) and step>7000:
        output_image(grid,f"output_{step}.txt")
    # chk = detect_trees(grid)
    # if chk>10:
    #     print(chk)
        



In [301]:
middle_r = nr//2
middle_c = nc // 2



In [302]:
q1 = grid[:middle_r,:middle_c].sum() # upper left
q2 = grid[middle_r+1:,:middle_c].sum() # lower left
q3 = grid[:middle_r,middle_c+1:].sum() # upper right
q4 = grid[middle_r+1:,middle_c+1:].sum() # lower right
print(np.prod([q1,q2,q3,q4]))

229839456.0
